In [ ]:
import pandas as pd
from typing import List, Optional, Any, Iterable, Literal, Tuple
from pydantic import BaseModel, Field
import logging
from openai import OpenAI
from utils import _retry_call, _get_structured_response, _parse_formatted_question
from report_functions import _get_q_type, _create_pivot_for_question, _process_one_filter, find_top_matches

import dotenv
import os

dotenv.load_dotenv()
API_KEY = os.getenv("OR_API_KEY")

## Подготовка вопросов

In [ ]:
# def tranc_unique(x: set|list|pd.Series, thr: int = 15):
#     if isinstance(x, pd.Series):
#         unique = sorted(x.dropna().unique())
#     elif isinstance(x, list):
#         unique = sorted(set(x))
#     elif isinstance(x, set):
#         unique = sorted(x)
#     else:
#         raise ValueError
#     if len(unique) > thr:
#         return unique[:thr] + ["..."]
#     return unique

# df = pd.read_parquet(r"D:\WORK\db_nomgnt_latest.parquet", engine="fastparquet")

Фильтр по волне

In [ ]:
# wave = "2025-03"

# df = df[df["wave"] == wave]
# df["question"].drop_duplicates().to_excel(f"questions_from_{wave}.xlsx", index=False)

Фильтр по году начала

In [ ]:
# YEAR = 2020

# waves = df["wave"].cat.categories.to_series()
# years = waves.str.split("-").str.get(0).astype(int)
# latest_waves = years[years >= YEAR].index

# df = df[df["wave"].isin(latest_waves)]

# QAs = _parse_formatted_question(df["question"].cat.categories.to_series(name="question"))
# ans = df.groupby("question", observed=True).agg(
#     answers=("answer", lambda x: set(x.dropna()))
# )
# QAs = pd.concat([QAs, ans], axis=1, join="inner")

# QAs = QAs.groupby("q_clean", observed=True).agg(
#     details=("detail", lambda x: tranc_unique(x, 30)),
#     options=("option", lambda x: tranc_unique(x, 30)),
#     answers=("answers", lambda x: tranc_unique(set().union(*x)))
# ).reset_index()

# QAs.to_excel(f"questions_from_{YEAR}.xlsx", index=False)

## Ключ, модель и путь к вопросам

In [2]:
qs_path = "questions_from_2025-03.xlsx"

qs = pd.read_excel(qs_path)
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=API_KEY,
)

## Функции

In [3]:
# Pydantic-схемы результата

class ScoredQuestion(BaseModel):
    question: str = Field(..., description="Точная формулировка вопроса из базы")
    reason: str = Field(..., description="Почему этот вопрос полезен для ответа на запрос")
    relevance: float = Field(..., description="Оценка релевантности 0–100")

class RankedQuestions(BaseModel):
    results: List[ScoredQuestion] = Field(
        default_factory=list,
        description="Список релевантных вопросов с объяснениями и оценками"
    )

In [4]:
logger = logging.getLogger(__name__)

def extract_questions(
    client: OpenAI,
    model: str,
    user_query: str,
    all_questions: List[str],
    temperature: float = 1.2,
    retries: int = 3,
    base_delay: float = 1.0,
) -> str:
    if not all_questions:
        return "Список вопросов пуст — выбирать нечего."

    questions_block = "\n".join(f"{i+1}. {q}" for i, q in enumerate(all_questions))

    prompt = prompt = f"""
Запрос пользователя: {user_query}

Список доступных вопросов:
{questions_block}

---
Инструкция:

Проанализируй запрос и подбери релевантные вопросы из списка с оценкой релевантности (0-100).

Формат вывода:

**Рассуждение:**
[Краткий анализ запроса: 2-3 предложения о том, что нужно пользователю]

**Рекомендованные вопросы:**

1. **[92/100]** "[Исходная формулировка вопроса]"
   • **Обоснование:** [Почему вопрос напрямую отвечает на запрос]

2. **[78/100]** "[Исходная формулировка вопроса]"
   • **Обоснование:** [Как вопрос связан с ключевой темой]

3. **[65/100]** "[Исходная формулировка вопроса]"
   • **Обоснование:** [Какой контекст или смежную тему раскрывает]

4. **[55/100]** "[Исходная формулировка вопроса]"
   • **Обоснование:** [Чем может быть полезен для понимания]
---
"""

    def _call():
        resp = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt},
            ],
            temperature=temperature,
        )
        return resp.choices[0].message.content.strip()

    return _retry_call(_call, retries=retries, base_delay=base_delay)

def unite_questions(
    client: OpenAI,
    model: str,
    extraction_text: str,
    all_questions: List[str],
    max_keep: Optional[int],
    temperature: float = 1,
    retries: int = 3,
    base_delay: float = 1.0,
) -> RankedQuestions:
    """
    Принимает текст из extract_questions и ПОЛНЫЙ список формулировок.
    Возвращает строго структурированный список (Pydantic) с точными формулировками из базы.

    Требуется модель с поддержкой native structured output (Responses API -> parse).
    """
    if not all_questions:
        return RankedQuestions(results=[])

    questions_block = "\n".join(f"{i+1}. {q}" for i, q in enumerate(all_questions))
    cap = "Выбери столько, сколько действительно нужно."
    if isinstance(max_keep, int) and max_keep > 0:
        cap = f"Максимум {max_keep} штук."

    prompt = f"""Твоя задача — сопоставить результаты предыдущего шага с ТЕМИ ЖЕ точными формулировками из базы.

ВХОД:
- Текстовый отчёт предыдущего шага (с рекомендациями и оценками):
\"\"\"{extraction_text}\"\"\"

- Полный список ДОПУСТИМЫХ (ТОЧНЫХ) формулировок вопросов — выбор разрешён ТОЛЬКО из него:
{questions_block}

ТРЕБОВАНИЯ:
- Поле results — список уникальных элементов.
- Для каждого элемента:
  - question — ТОЧНАЯ формулировка из списка выше (обязательно одно из перечисленных).
  - reason — сжато, своими словами, почему он уместен.
  - relevance — число 0–100 (можно скорректировать оценки из отчёта).
- {cap}
- Если соответствия неочевидны, выбери наиболее близкую по смыслу формулировку ИЗ СПИСКА.
- Никаких формулировок вне списка.

Верни только структурированный объект.
"""

    parsed: RankedQuestions = _get_structured_response(
        client=client,
        model=model,
        prompt=prompt,
        response_model=RankedQuestions,
        temperature=temperature,
        retries=retries,
        base_delay=base_delay,
    )

    # Валидация ответов
    valid_set = set(all_questions)
    unique = []
    seen = set()
    for item in parsed.results:
        q = (item.question or "").strip()
        if not q or q not in valid_set or q in seen:
            logger.warning(f"Пропущено: '{q[:60]}...' (неточного соответствия или дубликат)")
            continue
        seen.add(q)
        # нормируем диапазон на всякий случай
        score = max(0.0, min(100.0, float(item.relevance)))
        unique.append(ScoredQuestion(question=q, reason=item.reason.strip(), relevance=score))

    return RankedQuestions(results=unique)


## Проверка

In [5]:
questions = qs["question"].to_list()
print(f"Вопросов: {len(questions)}")

Вопросов: 1384


### Уточняем по запросу пользователя какие вопросы нужны

### Вытаскиваем вопросы в свободной форме

In [6]:
user_query = "Посчитай мне размер средних сбережений и норму сбережний среди тех респондентов, у которых они есть."

extract_text = extract_questions(
    client,
    model="tngtech/deepseek-r1t2-chimera:free",
    user_query=user_query,
    all_questions=questions,
)
print(extract_text)

**Рассуждение:**  
Пользователю требуется рассчитать средний размер сбережений и норму сбережений среди респондентов, у которых такие сбережения есть. Нужны вопросы, содержащие данные о наличии сбережений, их величине и доле сберегаемого дохода.

**Рекомендованные вопросы:**

1. **[44] [95/100]** "[N3] Как бы Вы охарактеризовали величину сбережений Вашей семьи?"  
   • **Обоснование:** Напрямую оценивает качественные характеристики сбережений (например, "маленькие", "большие"), что можно использовать для категоризации респондентов, *у кого сбережения есть*.

2. **[45] [100/100]** "[A26] Каков размер денежных сбережений Вашей семьи (наличные деньги, депозиты в банках, ценные бумаги)?"  
   • **Обоснование:** Прямой вопрос о количественном размере сбережений. Незаменим для расчета средних значений среди тех, у кого есть сбережения.

3. **[46] [95/100]** "[N4] Какой % от Вашего дохода Вы сберегаете ежемесячно?"  
   • **Обоснование:** Замеряет норму сбережений — ключевой показатель для за

### Вытаскиваем вопросы с жесткой привязкой к модели Pydantic

In [9]:
ranked = unite_questions(
    client,
    model="meta-llama/llama-4-maverick:free",
    # model="meta-llama/llama-3.3-8b-instruct:free",
    extraction_text=extract_text,
    all_questions=questions,
    max_keep=100
)
print(ranked.model_dump())

{'results': [{'question': '[N3] Как бы Вы охарактеризовали величину сбережений Вашей семьи?', 'reason': 'Прямая оценка качественных характеристик сбережений', 'relevance': 95.0}, {'question': '[A26] Каков размер денежных сбережений Вашей семьи (наличные деньги, депозиты в банках, ценные бумаги)?', 'reason': 'Прямая количественная оценка сбережений', 'relevance': 100.0}, {'question': '[N4] Какой % от Вашего дохода Вы сберегаете ежемесячно?', 'reason': 'Оценка нормы сбережений', 'relevance': 95.0}, {'question': '[Q16] К какому экономическому классу Вы себя относите?', 'reason': 'Косвенная оценка социально-экономического статуса', 'relevance': 65.0}, {'question': '[J6] Как часто Вы покупаете продукты питания (без учета онлайн-заказов)?', 'reason': 'Оценка паттернов экономии на повседневных расходах', 'relevance': 55.0}]}


In [10]:
chosen_qs = [sq.question for sq in ranked.results]
chosen_qs

['[N3] Как бы Вы охарактеризовали величину сбережений Вашей семьи?',
 '[A26] Каков размер денежных сбережений Вашей семьи (наличные деньги, депозиты в банках, ценные бумаги)?',
 '[N4] Какой % от Вашего дохода Вы сберегаете ежемесячно?',
 '[Q16] К какому экономическому классу Вы себя относите?',
 '[J6] Как часто Вы покупаете продукты питания (без учета онлайн-заказов)?']

## Tool для LLM

In [11]:
df_path = r"D:\WORK\db_nomgnt_latest.parquet"
ext_path = r"D:\WORK\data\EXT\EXT_data.xlsx"

df = pd.read_parquet(df_path, engine="fastparquet")
df_QS = df["question"].cat.categories.to_frame(name="question")
df_QS[["type", "option"]] = _get_q_type(df_QS["question"])

df_ext = pd.read_parquet(ext_path, engine="fastparquet")

In [12]:
def _select_questions(
    df_QS: pd.DataFrame,
    queries: str | Iterable[str],
    *,
    exact: bool = False,
) -> pd.DataFrame:
    """Возвращает подтаблицу df_QS с вопросами, подходящими под шаблон(ы).

    - queries: один шаблон или список шаблонов. Если exact=False, то используется contains (без regex),
      иначе — строгие совпадения по "question".
    """
    if isinstance(queries, str):
        queries = [queries]
    queries = list(queries)

    parts: list[pd.DataFrame] = []
    for q in queries:
        if exact:
            parts.append(df_QS[df_QS["question"].isin([q])])
        else:
            parts.append(df_QS[df_QS["question"].str.contains(q, regex=False)])
    if not parts:
        return df_QS.iloc[0:0]
    out = pd.concat(parts, axis=0).drop_duplicates(ignore_index=True)
    return out

def _combine_filters(
    DB: pd.DataFrame,
    df_QS: pd.DataFrame,
    df_ext: pd.DataFrame | None,
    filters: Iterable[Tuple[str, str, Literal["AND", "OR"]]] | None,
    waves: Iterable[str] | None,
    *,
    verbose: int = 1,
) -> tuple[pd.DataFrame | None, list[Tuple[str, str, str]]]:
    """Повторяет логику _filter_df, но принимает структурированный список фильтров.

    filters: список троек (question, value|"CHECKED"|TOWNSIZE_value, logic)
    waves: если передан, то отфильтровывает DB по волнам ДО применения фильтров по вопросам.
    Возвращает (filtered_df | None, applied_filters | []).
    """
    if filters is None:
        filters = []
    filters = list(filters)

    if waves is not None:
        waves = list(waves)
        DB = DB[DB["wave"].isin(waves)]

    if not filters:
        return DB, []

    mask = pd.Series(True, index=DB.index)
    problem_flag = False
    applied: list[Tuple[str, str, str]] = []

    for q, v, t in filters:
        cur_mask = _process_one_filter(DB, df_QS, df_ext, q, v, verbose)
        if cur_mask is None:
            problem_flag = True
            continue
        if t == "AND":
            mask &= cur_mask
        elif t == "OR":
            mask |= cur_mask
        else:
            problem_flag = True
            if verbose >= 1:
                print(f"!!! Допустимые значения операций: AND, OR (дано - '{t}')")
            continue
        applied.append((q, v, t))

    if problem_flag:
        return None, []

    if mask.sum() == 0:
        if verbose >= 1:
            print("!!! По указанным фильтрам нет респондентов")
        return None, []

    uids = DB.loc[mask, "respondent_uid"].unique().tolist()
    return DB.loc[DB["respondent_uid"].isin(uids)], applied

def _process_crosstab(
    DB_filtered: pd.DataFrame,
    df_QS: pd.DataFrame,
    rows_qs: pd.DataFrame,
    cols_qs: pd.DataFrame,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Кросстаб по двум вопросам (строки × столбцы).

    Поведение процентовки аналогично _process_q: проценты считаются по столбцам,
    знаменатель – число уникальных респондентов, давших любой ответ по столбцовому вопросу.
    """
    # Проверка типов
    if rows_qs["type"].nunique() > 1:
        raise ValueError("Найденные вопросы для строк имеют разные типы (MULTI/SINGLE/MIX)")
    if cols_qs["type"].nunique() > 1:
        raise ValueError("Найденные вопросы для столбцов имеют разные типы (MULTI/SINGLE/MIX)")

    row_type = rows_qs["type"].iloc[0] if not rows_qs.empty else None
    col_type = cols_qs["type"].iloc[0] if not cols_qs.empty else None

    if row_type is None or col_type is None:
        raise ValueError("Пустой набор вопросов для строк или столбцов")

    row_val_col = "option" if row_type == "MULTI" else "answer"
    col_val_col = "option" if col_type == "MULTI" else "answer"

    df_row = DB_filtered[DB_filtered["question"].isin(rows_qs["question"])]
    df_col = DB_filtered[DB_filtered["question"].isin(cols_qs["question"])]

    # Минимизация и устранение дублей на уровне (respondent_uid, wave, значение)
    row_slim = (
        df_row[["respondent_uid", "wave", row_val_col]]
        .dropna()
        .drop_duplicates()
        .rename(columns={row_val_col: "row_val"})
    )
    col_slim = (
        df_col[["respondent_uid", "wave", col_val_col]]
        .dropna()
        .drop_duplicates()
        .rename(columns={col_val_col: "col_val"})
    )

    # Соединяем по респонденту и волне, получая все пары значений
    pairs = pd.merge(row_slim, col_slim, on=["respondent_uid", "wave"], how="inner")

    # Считаем уникальных респондентов в каждой ячейке
    pivot = pd.pivot_table(
        pairs,
        index="row_val",
        columns="col_val",
        values="respondent_uid",
        aggfunc="nunique", # type: ignore
        fill_value=0,
        observed=True,
    ) # type: ignore

    # Добавляем строку "Всего" по столбцам.
    denom_by_col = col_slim.groupby("col_val")["respondent_uid"].nunique()
    # Переупорядочим и заполним нулями для отсутствующих столбцов
    denom_by_col = denom_by_col.reindex(pivot.columns, fill_value=0)
    pivot.loc["Всего", :] = denom_by_col
    pivot.index.name = "Ответы"

    # Проценты – как share по столбцам (без строки "Всего")
    pivot_pct = pivot.drop(index="Всего").divide(denom_by_col, axis=1)

    return pivot, pivot_pct

def build_pivot_tool(
    DB: pd.DataFrame,
    *,
    # Строки сводной таблицы: один шаблон или список; exact=False трактует как contains
    rows_query: str | Iterable[str],
    rows_exact: bool = False,
    # Фильтры вида [(question, value|"CHECKED", "AND"|"OR"), ...]
    filters: Iterable[Tuple[str, str, Literal["AND", "OR"]]] | None = None,
    # Доп. фильтрация по волнам (актуальна и для columns="cross")
    waves: Iterable[str] | None = None,
    # Столбцы сводной: "wave" (по умолчанию) или "cross"
    columns: Literal["wave", "cross"] = "wave",
    # Для columns="cross" – вопрос(ы) для столбцов
    cols_query: str | Iterable[str] | None = None,
    cols_exact: bool = False,
    # Данные для кастом-фильтров (например, TOWNSIZE)
    df_ext: pd.DataFrame | None = None,
    # Управление форматом возврата
    return_pct: bool = True,
    as_json: bool = False,
    verbose: int = 1,
) -> dict[str, Any]:
    # Служебная таблица вопросов и их типов
    df_QS = DB["question"].cat.categories.to_frame(name="question")
    df_QS[["type", "option"]] = _get_q_type(df_QS["question"])  # re-use существующую функцию

    # Комбинируем фильтры
    DB_filtered, applied_filters = _combine_filters(DB, df_QS, df_ext, filters, waves, verbose=verbose)
    if DB_filtered is None:
        return {"pivot": pd.DataFrame(), "pivot_pct": pd.DataFrame() if return_pct else None, "meta": {"applied_filters": [], "rows": [], "cols": [], "waves": list(waves) if waves is not None else None}}

    # Выбор вопросов для строк
    rows_qs = _select_questions(df_QS, rows_query, exact=rows_exact)
    if rows_qs.empty:
        raise ValueError("НЕ найдено вопросов, подходящих под rows_query")

    meta: dict[str, Any] = {
        "applied_filters": applied_filters,
        "rows": rows_qs["question"].tolist(),
        "waves": list(waves) if waves is not None else None,
        "mode": columns,
    }

    if columns == "wave":
        # Логика полностью делегируется _process_q (максимальное переиспользование кода)
        if rows_qs["type"].nunique() > 1:
            raise ValueError("Найденные вопросы для строк имеют разные типы (MULTI/SINGLE/MIX)")
        q_type = "SINGLE" if rows_qs.shape[0] == 1 else rows_qs["type"].iloc[0]
        df_for_rows = DB_filtered[DB_filtered["question"].isin(rows_qs["question"])]
        pivot, pivot_pct = _create_pivot_for_question(df_for_rows, df_QS, q_type)
    else:
        # columns == "cross"
        if cols_query is None:
            raise ValueError("Для columns='cross' необходимо указать cols_query")
        cols_qs = _select_questions(df_QS, cols_query, exact=cols_exact)
        if cols_qs.empty:
            raise ValueError("НЕ найдено вопросов, подходящих под cols_query")
        meta["cols"] = cols_qs["question"].tolist()
        pivot, pivot_pct = _process_crosstab(DB_filtered, df_QS, rows_qs, cols_qs)

    if not return_pct:
        pivot_pct = None

    if as_json:
        out = {
            "pivot": pivot.reset_index().to_dict(orient="records"),
            "pivot_pct": None if pivot_pct is None else pivot_pct.reset_index().to_dict(orient="records"),
            "meta": meta,
        }
    else:
        out = {"pivot": pivot, "pivot_pct": pivot_pct, "meta": meta}

    return out

In [62]:
tool = {
  "type": "function",
  "function": {
    "name": "build_pivot",
    "description": "Построение сводной таблицы из данных опросов с фильтрацией и группировкой. Позволяет анализировать ответы респондентов по различным вопросам, применять фильтры и создавать кросстабуляции.",
    "parameters": {
      "type": "object",
      "properties": {
        "rows_query": {
          "description": "Название вопроса(ов) для строк сводной таблицы. По умолчанию используется подстрока, если rows_exact=false",
          "anyOf": [
            {"type": "string"},
            {"type": "array", "items": {"type": "string"}}
          ]
        },
        "rows_exact": {
          "type": "boolean",
          "description": "Если true, точное совпадение названия вопроса. Если false, поиск по подстроке",
          "default": False
        },
        "filters": {
          "type": "array",
          "items": {
            "type": "object",
            "properties": {
              "question": {
                "type": "string",
                "description": "Вопрос или его часть для фильтрации"
              },
              "value": {
                "type": "string",
                "description": "Значение ответа для фильтра, или 'CHECKED' для множественного выбора"
              },
              "logic": {
                "type": "string",
                "enum": ["AND", "OR"],
                "description": "Логика комбинирования фильтров: AND или OR"
              }
            },
            "required": ["question", "value", "logic"]
          },
          "description": "Список фильтров для применения. Каждый фильтр задает вопрос, значение и логику AND/OR"
        },
        "waves": {
          "type": "array",
          "items": {"type": "string"},
          "description": "Фильтрация данных по конкретным волнам (например, ['2024-01', '2025-01'])"
        },
        "columns": {
          "type": "string",
          "enum": ["wave", "cross"],
          "description": "Режим столбцов: 'wave' группирует по волнам, 'cross' создает кросстаб с cols_query",
          "default": "wave"
        },
        "cols_query": {
          "description": "Название вопроса(ов) для столбцов при columns='cross'. Обязательно для режима кросстабуляции",
          "anyOf": [
            {"type": "string"},
            {"type": "array", "items": {"type": "string"}}
          ]
        },
        "cols_exact": {
          "type": "boolean",
          "description": "Если true, точное совпадение cols_query. Если False, поиск по подстроке",
          "default": False
        }
      },
      "required": ["rows_query"]
    }
  }
}

tools = [tool]

In [ ]:
# model = "meta-llama/llama-3.3-70b-instruct:free"
model = "meta-llama/llama-4-maverick:free"

messages = [
    {
        "role": "user",
        "content": (
            f"Ты — аналитик данных. В твоем распоряжении есть база данных ответов респондентов на различные вопросы. "
            f"Тебе дан пользовательский запрос - '{user_query}'.\n"
            "У тебя также есть контекст:\n\n{extract_text}\n\n"
            "Твоя задача - пользуясь ТОЛЬКО контекстом, а также указанными инструментами, построить возможный план анализа. "
            f"Точные формулировки допустимых вопросов для анализа - {chosen_qs}"
        )
    }
]

response = client.chat.completions.create(
    model=model,
    messages=messages, # type: ignore
    tools=tools, # type: ignore
    tool_choice="auto",
    max_tokens=4_000,
    temperature=0.2
)

plan = response.choices[0].message.content

In [40]:
print(plan)

Для выполнения запроса "Посчитай мне размер средних сбережений и норму сбережений среди тех респондентов, у которых они есть" на основе предоставленного контекста и доступных вопросов для анализа, можно составить следующий план:

1. **Определить наличие сбережений**: Для начала необходимо определить, у каких респондентов есть сбережения. Это можно сделать с помощью вопроса '[A26] Каков размер денежных сбережений Вашей семьи (наличные деньги, депозиты в банках, ценные бумаги)?'. Респонденты с ненулевым размером сбережений считаются имеющими сбережения.

2. **Фильтрация данных**: Отфильтровать данные, чтобы включить только респондентов с ненулевыми сбережениями.

3. **Расчет среднего размера сбережений**: Используя отфильтрованные данные, рассчитать средний размер сбережений с помощью вопроса '[A26] Каков размер денежных сбережений Вашей семьи (наличные деньги, депозиты в банках, ценные бумаги)?'.

4. **Расчет нормы сбережений**: Для расчета нормы сбережений использовать вопрос '[N4] Как

In [63]:
# model = "meta-llama/llama-4-maverick:free"
model = "meta-llama/llama-3.3-70b-instruct:free"

questions_block = "\n".join(f"{i+1}. {q}" for i, q in enumerate(chosen_qs))

messages = [
    {
        "role": "user",
        "content": (
            f"Ты — агент по вызову функций. У тебя есть составленный план:\n{plan}\n"
            "Соверши указанные в нем вызовы функций, которые тебе доступны. "
            f"Точные формулировки доступных вопросов:\n{questions_block}"
            "\nПроверяй JSON на валидность!"
        )
    }
]

response = client.chat.completions.create(
    model=model,
    messages=messages, # type: ignore
    tools=tools, # type: ignore
    tool_choice="auto",
    temperature=0.2
)

In [65]:
print(response.choices[0].message)

ChatCompletionMessage(content='\n 1}</function>', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_c0ahlELgpsxxvOCECRNyo3mM', function=Function(arguments='{"rows_query": "[A26] \\u041a\\u0430\\u043a\\u043e\\u0432 \\u0440\\u0430\\u0437\\u043c\\u0435\\u0440 \\u0434\\u0435\\u043d\\u0435\\u0436\\u043d\\u044b\\u0445 \\u0441\\u0431\\u0435\\u0440\\u0435\\u0436\\u0435\\u043d\\u0438\\u0439 \\u0412\\u0430\\u0448\\u0435\\u0439 \\u0441\\u0435\\u043c\\u044c\\u0438 (\\u043d\\u0430\\u043b\\u0438\\u0447\\u043d\\u044b\\u0435 \\u0434\\u0435\\u043d\\u044c\\u0433\\u0438, \\u0434\\u0435\\u043f\\u043e\\u0437\\u0438\\u0442\\u044b \\u0432 \\u0431\\u0430\\u043d\\u043a\\u0430\\u0445, \\u0446\\u0435\\u043d\\u043d\\u044b\\u0435 \\u0431\\u0443\\u043c\\u0430\\u0433\\u0438)?", "filters": [{"query": "[A26] \\u041a\\u0430\\u043a\\u043e\\u0432 \\u0440\\u0430\\u0437\\u043c\\u0435\\u0440 \\u0434\\u0435\\u043d\\u0435\\u0436\\u043d\

In [66]:
FUNCTIONS_MAPPER = {
    "build_pivot": build_pivot_tool
}

In [67]:
import json

msg = response.choices[0].message

if msg.tool_calls:
    fn = msg.tool_calls[0].function     # Только первый tool call
    if fn.name in FUNCTIONS_MAPPER:
        called_fn = FUNCTIONS_MAPPER[fn.name]
        print(fn.arguments)
        args = json.loads(fn.arguments or "{}")

        filters = None
        if args.get("filters"):
            filters = [
                (f["query"], f["value"], f["logic"]) 
                for f in args["filters"]
            ]
        
        res = called_fn(
            df,
            rows_query=args["rows_query"],
            columns=args.get("columns", "wave"),
            filters=filters,
            waves=args.get("waves"),
            cols_query=args.get("cols_query"),
            rows_exact=args.get("rows_exact", False),
            cols_exact=args.get("cols_exact", False),
            return_pct=args.get("return_pct", True),
            as_json=True,
            verbose=args.get("verbose", 1),
        )

{"rows_query": "[A26] \u041a\u0430\u043a\u043e\u0432 \u0440\u0430\u0437\u043c\u0435\u0440 \u0434\u0435\u043d\u0435\u0436\u043d\u044b\u0445 \u0441\u0431\u0435\u0440\u0435\u0436\u0435\u043d\u0438\u0439 \u0412\u0430\u0448\u0435\u0439 \u0441\u0435\u043c\u044c\u0438 (\u043d\u0430\u043b\u0438\u0447\u043d\u044b\u0435 \u0434\u0435\u043d\u044c\u0433\u0438, \u0434\u0435\u043f\u043e\u0437\u0438\u0442\u044b \u0432 \u0431\u0430\u043d\u043a\u0430\u0445, \u0446\u0435\u043d\u043d\u044b\u0435 \u0431\u0443\u043c\u0430\u0433\u0438)?", "filters": [{"query": "[A26] \u041a\u0430\u043a\u043e\u0432 \u0440\u0430\u0437\u043c\u0435\u0440 \u0434\u0435\u043d\u0435\u0436\u043d\u044b\u0445 \u0441\u0431\u0435\u0440\u0435\u0436\u0435\u043d\u0438\u0439 \u0412\u0430\u0448\u0435\u0439 \u0441\u0435\u043c\u044c\u0438 (\u043d\u0430\u043b\u0438\u0447\u043d\u044b\u0435 \u0434\u0435\u043d\u044c\u0433\u0438, \u0434\u0435\u043f\u043e\u0437\u0438\u0442\u044b \u0432 \u0431\u0430\u043d\u043a\u0430\u0445, \u0446\u0435\u043d\u043d\u0

In [60]:
res

{'pivot': Empty DataFrame
 Columns: []
 Index: [],
 'pivot_pct': None,
 'meta': {'applied_filters': [], 'rows': [], 'cols': [], 'waves': None}}